<a href="https://colab.research.google.com/github/lucasangio01/amazon-books/blob/master/notebook_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

KeyboardInterrupt: 

In [ ]:
"""
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!pip install kaggle
!kaggle datasets download -d mohamedbakhet/amazon-books-reviews
!unzip amazon-books-reviews.zip
"""

!apt-get install openjdk-11-jdk -y
!pip install pandas numpy pyspark sparknlp spark-nlp==5.3.3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxt-dev libxtst6 libxxf86dga1 openjdk-11-jre
  x11-utils
Suggested packages:
  libxt-doc openjdk-11-demo openjdk-11-source visualvm mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxt-dev libxtst6 libxxf86dga1 openjdk-11-jdk
  openjdk-11-jre x11-utils
0 upgraded, 10 newly installed, 0 to remove and 35 not upgraded.
Need to get 6,920 kB of archives.
After this operation, 16.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-core all 2.37-2build1 [1,041 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-extra all 2.37-2build1 [2,041 kB]
Get:3 http://archive.ubuntu.com/ubuntu jam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, when, from_unixtime, length, year, col, array_size, udf, avg, collect_list, row_number
from pyspark.sql.types import ArrayType, FloatType, StructType, StructField, IntegerType, StringType
from pyspark.sql.window import Window

from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, IDF, BucketedRandomProjectionLSH, Normalizer
from pyspark.ml.linalg import Vectors, VectorUDT, DenseVector

from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, StopWordsCleaner, LemmatizerModel, SentenceDetectorDLModel, NorvigSweetingModel, BertSentenceEmbeddings



---


# Utils.py file:

In [ ]:
# some books have duplicates, with some differences in the title. We only keep one of each
duplicated_titles = ["Pride & Prejudice (Penguin Classics)", "Pride & Prejudice (New Windmill)", "Hannibal (Hannibal Lecter)", "Jane Eyre (Penguin Classics)", "Jane Eyre (Large Print)",
                      "Jane Eyre (Signet classics)", "Jane Eyre (Simple English)", "Jane Eyre (New Windmill)", "Jane Eyre (Everyman's Classics)", "Jane Eyre: Complete and Unabridged (Puffin Classics)",
                      "Of Mice and Men (Penguin Audiobooks)", "Me Talk Pretty One Day (Turtleback School & Library Binding Edition)", "Me Talk Pretty One Day C", "Wuthering Heights (Riverside editions)",
                      "Wuthering Heights (Penguin Audiobooks)", "Wuthering Heights (New Windmill)", "A Tale of Two Cities, Literary Touchstone Edition",
                      "A Christmas Carol, in Prose: Being a Ghost Story of Christmas (Collected Works of Charles Dickens)", "Christmas Carol (Ladybird Classics)", "Frankenstein (Running Press classics)",
                      "Signature Classics - Great Expectations (Signature Classics Series)", "The Scarlet Letter (Lake Illustrated Classics, Collection 2)", "The Adventures of Huckleberry Finn (Courage Literary Classics)",
                      "The Picture of Dorian Gray (The Classic Collection)", "Picture of Dorian Gray", "Little Women (Courage giant classics)", "Down Under; Abridged", "Heart of Darkness (Everyman Classics)",
                      "The Secret Garden (Worlds Classics)", "everything on this page is for Treasure Island", "In the Heart of the Sea", "the Picture of Dorian Gray", "Wuthering Heights.", "Daughter of Fortune CD",
                      "Adventures of Huckleberry Finn (Simple English)", "Awakening: Kate Chopin Pb (Case Studies in Contemporary)", "Tess of the D'Urbervilles (New Wessex editions)",
                      "The Awakening: Complete, Authoritative Text With Biographical and Historical Contexts, Critical History, and Essays from Five Contemporary Critical Perspectives (Case Studies in Contemporary Criticism)",
                      "Tess of the d'Urbervilles (Cambridge Literature)", "Tess of Th D'Urbervilles (Pbk)(Oop) (Bloom's Notes)", "Tess of the Durbervilles", "Tess of the D'urbervilles (Summer Classics)",
                      "The Heart Is a Lonely Hunter", "Stone Of Tears (Turtleback School & Library Binding Edition) (Sword of Truth)", "Lucky Man", "The Call of the Wild (Dover Large Print Classics)",
                      "Sense And Sensibility (CH) (Jane Austen Collection)", "Sense and Sensibility (Wordsworth Hardback Library)", "Sense and sensibility", "Emma (Signet classics)", "Emma (Riverside Editions)",
                      "Emma (CH) (Jane Austen Collection)", "Emma (Progress English)", "Emma (The World's Classics)", "Emma (Radio Collection)", "Emma (Summer Classics)", "Persuasion (World's Classics)",
                      "Bet Me (Brilliance Audio on Compact Disc)", "Sense & Sensibility Cds (Penguin Classics)", "Tess of the D'Urbervilles", "Wuthering Heights (Signet classics)", "Alice in Wonderland",
                      "Alice in Wonderland (Tell tales)", "The Picture of Dorian Gray (Classic Collection (Brilliance Audio))", "Great Expectations (Signet classics)", "Heart of Darkness and the Secret Sharer",
                      "Alice's Adventures in Wonderland and Through the Looking Glass (Classic Collection (Brilliance Audio))", "Treasure Island (Classic Illustrated)",
                      "Under and Alone: The True Story of the Undercover Agent Who Infiltrated America's Most Violent Outlaw Motorcycle Gang", "This Present Darkness (Turtleback School & Library Binding Edition)",
                      "Angela's Ashes (Turtleback School & Library Binding Edition)", "The Awakening: Complete, Authoritative Text With Biographical & Historical Contexts, Critical History, & Essays from Five Contemporary Critica. Perspectives (Case Studies in Contemporary Criticism)",
                      "Casting the First Stone", "The Hound of the Baskervilles (Signet Classics)", "Hound of the Baskervilles (Lrs Large Print Heritage Series)"]


def load_data(spark, chosen_data):

  duplicated_df = spark.createDataFrame([(title,) for title in duplicated_titles], ["book_title"])

  reviews_schema = StructType([
      StructField("Id", IntegerType(), True),
      StructField("Title", StringType(), True),
      StructField("Price", IntegerType(), True),
      StructField("User_id", StringType(), True),
      StructField("profileName", StringType(), True),
      StructField("review/helpfulness", StringType(), True),
      StructField("review/score", FloatType(), True),
      StructField("review/time", IntegerType(), True),
      StructField("review/summary", StringType(), True),
      StructField("review/text", StringType(), True)
      ])

  books_data_schema = StructType([
    StructField("Title", StringType(), True),
    StructField("description", StringType(), True),
    StructField("categories", StringType(), True)
    ])

  original_df = (spark.read.csv("../content/drive/MyDrive/Colab Notebooks/Books_rating.csv", header = True, schema = reviews_schema)
      .drop("Price", "profileName", "review/summary")
      .filter(length("review/text").between(300, 600))  # only keeping reviews between 300 and 600 characters
      .dropna()
      .withColumn("review_year", year(from_unixtime("review/time")))
      .drop("review/time")
      .withColumn("review/score", col("review/score").cast(IntegerType()))
      .withColumnRenamed("Id", "book_id")
      .withColumnRenamed("Title", "book_title")
      .withColumnRenamed("User_id", "user_id")
      .withColumnRenamed("review/helpfulness", "review_helpfulness")
      .withColumnRenamed("review/score", "review_score")
      .withColumnRenamed("review/text", "review_text")
      .cache()
      )

  reviews_per_book = 50

  if chosen_data == "big_only_fiction": # choose if you want to analyze only the "fiction" books or all the categories
    min_reviews = 50
  elif chosen_data == "big_full":   # choose if you want to analyze the books of all the categories
    min_reviews = 105
  elif chosen_data == "trial_full": # choose if you want to analyze a smaller subset of the data, for faster (but less informative) results
    min_reviews = 260

  famous_titles = original_df.select("book_title").groupBy("book_title").count().filter(col("count") > min_reviews).drop("count")    # select the books with a certain number of reviews
  df_reviews = original_df.select("book_title", "review_text").join(famous_titles, on="book_title", how="inner").join(duplicated_df, on="book_title", how="left_anti")
  original_df.unpersist()

  """
  Bigger dataset: only fiction (6750 x 135), full (6750 x 135)
  Trial dataset: full(800 x 16)
  """

  if chosen_data == "big_only_fiction":  # choose if you want to analyze only the "fiction" books
    titles_list = df_reviews.select("book_title").distinct()
    books_data_full = spark.read.csv("../content/drive/MyDrive/Colab Notebooks/books_data.csv", inferSchema = True, header = True).select("Title", "categories", "description").withColumnRenamed("Title", "book_title").dropna().filter(col("categories") == "['Fiction']").join(titles_list, on="book_title", how="inner").cache()
    books_with_description = books_data_full.select("book_title", "description").cache()

  window_spec = Window.partitionBy("book_title").orderBy("review_text")
  df_reviews = df_reviews.withColumn("row_num", row_number().over(window_spec)).filter(col("row_num") <= reviews_per_book) # select how many reviews to use for the analysis, for each book

  if chosen_data == "big_only_fiction":
    df_reviews = df_reviews.drop("row_num", "description").join(books_data_full, on = "book_title", how = "inner").cache()
    df_descriptions = books_data_full.select("book_title", "description").cache()
    return df_reviews, df_descriptions
  else:
    df_reviews = df_reviews.drop("row_num").cache()
    return df_reviews, None


In [ ]:
def embedding_pipeline(dataset):
  if dataset == df_reviews:
    text_column = "review_text"
  else:
    text_column = "desctiption"

  document_assembler = DocumentAssembler().setInputCol(text_column).setOutputCol("document")
  sentence_embeddings = BertSentenceEmbeddings().pretrained("sent_small_bert_L2_128", "en").setInputCols(["document"]).setOutputCol("embeddings")
  pipeline = Pipeline(stages = [document_assembler, sentence_embeddings])

  result = pipeline.fit(dataset).transform(dataset)
  extract_embedding = udf(lambda arr: arr[0].embeddings, ArrayType(FloatType()))
  result = result.withColumn("embedding_vector", extract_embedding("embeddings")).select("book_title", "embedding_vector")

  to_vector_udf = udf(lambda arr: Vectors.dense(arr), VectorUDT())
  result = result.withColumn("embedding_vector_ml", to_vector_udf("embedding_vector")).select("book_title", "embedding_vector_ml")

  return result

In [ ]:
def compute_similarity(dataset):

  def average_vectors(vectors):
      arrays = np.array([v.toArray() for v in vectors])
      avg_arr = np.mean(arrays, axis=0)
      return DenseVector(avg_arr)

  avg_udf = udf(average_vectors, VectorUDT())
  book_embeddings = dataset.groupBy("book_title").agg(collect_list("embedding_vector_ml").alias("embeddings_list")).withColumn("book_embedding", avg_udf("embeddings_list")).select("book_title", "book_embedding")

  normalizer = Normalizer(inputCol = "book_embedding", outputCol = "norm_embedding", p = 2.0)
  book_embeddings = normalizer.transform(book_embeddings)

  def reviews_similarity():
    if chosen_data == "trial_full":
      threshold = 0.3
    elif chosen_data in ("big_only_fiction", "big_full"):
      threshold = 0.6
    lsh = BucketedRandomProjectionLSH(inputCol="norm_embedding", outputCol="hashes", bucketLength=0.5, numHashTables=3)
    similarity_model = lsh.fit(book_embeddings)
    similar_books = similarity_model.approxSimilarityJoin(book_embeddings, book_embeddings, threshold, distCol="distance").filter("datasetA.book_title < datasetB.book_title").withColumn("cosine_similarity", (1-col("distance"))).selectExpr("datasetA.book_title as book1", "datasetB.book_title as book2", "cosine_similarity").orderBy("cosine_similarity", ascending = False)
    return similar_books

  books_similarity = reviews_similarity()
  return books_similarity



---

# Notebook:

In [ ]:
spark = (SparkSession.builder.appName("Books").config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.3").getOrCreate())

In [ ]:
"""
Bigger dataset: big_only_fiction (6750 x 135), big_full (6750 x 135)
Trial dataset: trial_full (800 x 16)
"""

chosen_data = "big_only_fiction"
df_reviews, df_descriptions = load_data(spark, chosen_data)

In [ ]:
df_descriptions.show()

+--------------------+--------------------+
|          book_title|         description|
+--------------------+--------------------+
|    Whitney, My Love|Let New York Time...|
|Black Rose (In th...|Drie vrouwen ontm...|
| Shopaholic & Sister|Financial journal...|
|Gap Creek: The St...|A New York Times ...|
|      House of Mirth|A black comedy of...|
|A Salty Piece of ...|It's not on any c...|
|Domain (Domain Tr...|A beautiful psych...|
|In This Mountain ...|In her seventh in...|
|Alice's Adventure...|"Alice in Wonderl...|
|The Lies of Locke...|They say that the...|
|The Mark: The Bea...|His Excellency Gl...|
|Weapons of Choice...|In a first instal...|
|Dr. Jekyll and Mr...|Everyone has a da...|
|  A Princess of Mars|A Virginia gentle...|
|Gods and Kings (C...|Gods and Kings is...|
|     The Good German|Set in Berlin jus...|
|A Common Life (Th...|  No Marketing Blurb|
|The Importance of...|The four great co...|
|Lord Of Chaos (Tu...|For use in school...|
|One Shot (Jack Re...|Six shots.

In [ ]:
books_number = df_reviews.groupBy("book_title").count()
reviews_number = df_reviews.count()
print(f"Number of rows (reviews): {reviews_number}\nNumber of books: {books_number.count()}")

Number of rows (reviews): 6750
Number of books: 135


In [ ]:
books_number.sort("book_title", ascending = True).show(150, truncate = False)

+------------------------------------------------------------------------------------------------+-----+
|book_title                                                                                      |count|
+------------------------------------------------------------------------------------------------+-----+
|A Breath of Snow and Ashes (Outlander)                                                          |50   |
|A Common Life (The Mitford Years, Book 6)                                                       |50   |
|A Farewell to Arms                                                                              |50   |
|A Hunger Like No Other (Immortals After Dark, Book 1)                                           |50   |
|A Kiss of Shadows (Meredith Gentry, Book 1)                                                     |50   |
|A Land Remembered                                                                               |50   |
|A Princess of Mars                                    

In [ ]:
"""
def run_pipeline(dataset):
  document_assembler = DocumentAssembler().setInputCol("review_text").setOutputCol("document")
  #language_detector = LanguageDetectorDL.pretrained().setInputCols("document").setOutputCol("language")
  sentencer = SentenceDetectorDLModel().pretrained("sentence_detector_dl", "en").setInputCols(["document"]).setOutputCol("sentences")
  tokenizer = Tokenizer().setInputCols(["sentences"]).setOutputCol("token")
  checker = NorvigSweetingModel.pretrained().setInputCols("token").setOutputCol("checked")
  stopwords_cleaner = StopWordsCleaner().pretrained("stopwords_iso", "en").setInputCols(["checked"]).setOutputCol("clean_tokens").setCaseSensitive(False)
  lemmatizer = LemmatizerModel.pretrained("lemma_ewt", "en").setInputCols(["clean_tokens"]).setOutputCol("lemma")
  #ngrammer =  NGramGenerator().setInputCols(["lemma"]).setOutputCol("ngrams").setN(3).setEnableCumulative(True).setDelimiter('_')
  finisher = Finisher().setInputCols(["lemma"]).setCleanAnnotations(True)

  pipeline = Pipeline(stages = [document_assembler, sentencer, tokenizer, checker, stopwords_cleaner, lemmatizer, finisher])
  model = pipeline.fit(dataset)
  result = model.transform(dataset)#.filter(array_size("finished_lemma") > 20)

  return result
"""

In [ ]:
"""
data_title_review = df.select("book_title", "review_text")
result = run_pipeline(data_title_review)
"""

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
spellcheck_norvig download started this may take some time.
Approximate size to download 4.2 MB
[OK!]
stopwords_iso download started this may take some time.
Approximate size to download 2.1 KB
[OK!]
lemma_ewt download started this may take some time.
Approximate size to download 251.9 KB
[OK!]


In [ ]:
"""
def tf_idf(lemmatized_data):
  hashing_tf = HashingTF(inputCol = "finished_lemma", outputCol = "raw_features")
  featurized_data = hashing_tf.transform(lemmatized_data)

  idf = IDF(inputCol = "raw_features", outputCol = "features")
  idf_model = idf.fit(featurized_data)
  rescaled_data = idf_model.transform(featurized_data)

  return rescaled_data
"""

In [ ]:
"""
tfidf_features = tf_idf(result)
tfidf_features.show()
"""

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

## Check cosine similarity of the books

In [ ]:
embedded_reviews = embedding_pipeline(df_reviews)
similar_reviews = compute_similarity(embedded_reviews)

print(f"\n--- SHOWING SIMILAR BOOKS BASED ON USERS REVIEWS ---\nChosen dataset: {chosen_data}\n")
similar_reviews.show(50, truncate = False)

sent_small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]

--- SHOWING SIMILAR BOOKS BASED ON USERS REVIEWS ---
Chosen dataset: big_only_fiction

+--------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+------------------+
|book1                                                                                 |book2                                                                                 |cosine_similarity |
+--------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+------------------+
|Lord Of Chaos (Turtleback School & Library Binding Edition) (Wheel of Time (Pb))      |The Fires Of Heaven (Turtleback School & Library Binding Edition) (Wheel of Time (Pb))|0.9382876290812805|
|Cerulean Sins (Anita Bl

In [ ]:
embedded_descriptions = embedding_pipeline(df_descriptions)
similar_descriptions = compute_similarity(embedded_descriptions)

print(f"\n--- SHOWING SIMILAR BOOKS BASED ON BOOK DESCRIPTION ---\nChosen dataset: {chosen_data}\n")
similar_descriptions.show(50, truncate = False)

sent_small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]

--- SHOWING SIMILAR BOOKS BASED ON BOOK DESCRIPTION ---
Chosen dataset: big_only_fiction

+----------------------------------------------------------------+---------------------------------------------------+------------------+
|book1                                                           |book2                                              |cosine_similarity |
+----------------------------------------------------------------+---------------------------------------------------+------------------+
|Black Rose (In the Garden Series)                               |Wish You Well                                      |0.7823112488939672|
|Dreamcatcher Movie-Tie In                                       |Wish You Well                                      |0.7503098752757094|
|Dreamcatcher Movie-Tie In                                       |Little Women                                 